<a href="https://colab.research.google.com/github/elkhayyat17/Mistral7b_pdf_chatting/blob/main/Chat_with_papers_Mistral_7b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q pypdf
!pip install -q python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.6/277.6 kB 5.4 MB/s eta 0:00:00


In [2]:
!pip install -q transformers


In [3]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install  llama-cpp-python --no-cache-dir

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 24.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.20-cp310-cp310-manylinux_2_35_x86_64.whl size=7138259 sha256=b071967e29b2dd0bce76923397a39c70710bac06c93dd3743114ad3429cdffcd
  Stored in directory: /tmp/pip-ephem-wheel-cache-xftrg4ff/wheels/ef/f2/d2/0becb03047a348d7bd9a5b91ec88f4654d6fa7d67ea4e84d43
Successfully built llama-cpp-python


In [4]:
!pip install -q llama-index


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 923.4/923.4 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.4/221.4 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.


In [1]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext


Download your papers

In [ ]:
#!wget url of paper/s
!wget

Move pdf to Data directory

In [12]:
!mkdir Data
#!mv "/content/file name" "/content/Data"
!mv "/content/RTOC-assessment -report-2022.pdf" "/content/Data"




mkdir: cannot create directory ‘Data’: File exists


In [2]:
documents = SimpleDirectoryReader("/content/Data/").load_data()

In [3]:
import torch

from llama_index.llms import LlamaCPP
from llama_index.llms.llama_utils import messages_to_prompt, completion_to_prompt
llm = LlamaCPP(
    # You can pass in the URL to a GGML model to download it automatically
    model_url='https://huggingface.co/TheBloke/Mistral-7B-Instruct-v0.1-GGUF/resolve/main/mistral-7b-instruct-v0.1.Q4_K_M.gguf',
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    model_path=None,
    temperature=0.1,
    max_new_tokens=256,
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=3900,
    # kwargs to pass to __call__()
    generate_kwargs={},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": -1},
    # transform inputs into Llama2 format
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True,
)


AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


In [11]:
!pip -q install langchain sentence-transformers

In [4]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index.embeddings import LangchainEmbedding

embed_model = LangchainEmbedding(
  HuggingFaceEmbeddings(model_name="thenlper/gte-large")
)


In [5]:
from llama_index import ServiceContext
service_context = ServiceContext.from_defaults(
    chunk_size=256,
    llm=llm,
    embed_model=embed_model
)

In [6]:
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

In [7]:
query_engine = index.as_query_engine()
response = query_engine.query("summarize the paper   ")

In [8]:
print(response)

 The paper discusses the advantages of solar energy technology as a renewable and clean energy source that is plentiful, cheaper costs, less maintenance, and environmentally friendly. It highlights the significance of this paper in raising public awareness and disseminating the culture of solar energy usage in daily life. The paper's primary aim is to ensure sustainable development by highlighting solar energy applications worldwide. The scope of the paper includes a summary of installed capacity and the application of solar energy, the role of solar energy in sustainable development and employment of renewable energy, the perspective of solar energy, and conclusions and recommendations for future work.


Moniter the metadate : file name and page number which the response in

In [9]:
import re
document_info = str(response.metadata)
resorce=str(re.findall(r"'page_label': '[^']*', 'file_name': '[^']*'", document_info))
print(resorce)

["'page_label': '477', 'file_name': 'zkac023.pdf'", "'page_label': '477', 'file_name': 'zkac023.pdf'"]


In [ ]:
#while True:
#  query=input()
#  response = query_engine.query(query)
#  print(response)

Enable text wrapping so we don't have to scroll horizontally and create a function to flush CUDA cache.
**NOTE `input()` function will not work with if you run this cell**

In [21]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))

get_ipython().events.register('pre_run_cell', set_css)

def clear_cache():
  if torch.cuda.is_available():
    torch.cuda.empty_cache()

In [22]:
print(response)

 The paper discusses the advantages of solar energy technology as a renewable and clean energy source that is plentiful, cheaper costs, less maintenance, and environmentally friendly. It highlights the significance of this paper in raising public awareness and disseminating the culture of solar energy usage in daily life. The paper's primary aim is to ensure sustainable development by highlighting solar energy applications worldwide. The scope of the paper includes a summary of installed capacity and the application of solar energy, the role of solar energy in sustainable development and employment of renewable energy, the perspective of solar energy, and conclusions and recommendations for future work.


simple GUI so can chat wiht model within

In [10]:
import panel as pn
import param
pn.extension()
class cbfs(param.Parameterized):
    chat_history = param.List([])
    answer = param.String("")
    db_query  = param.String("")
    db_response = param.List([])

    def __init__(self,  **params):
        super(cbfs, self).__init__( **params)
        self.panels = []



    def convchain(self, query):
        if not query:
            return pn.WidgetBox(pn.Row('User:', pn.pane.Markdown("", width=600)), scroll=True)
        response = query_engine.query(query)
        self.answer =(response.response)

        self.panels.extend([
            pn.Row('User:', pn.pane.Markdown(query, width=600),styles={'background-color': '#F6F6F6'}),
            pn.Row('ChatBot:', pn.pane.Markdown(self.answer, width=600, styles={'background-color': '#F6F6F6'}))
        ])
        inp.value = ''  #clears loading indicator when cleared
        return pn.WidgetBox(*self.panels,scroll=True)



Note scrolling dose not wrok in colab so if u can not see chats rerun the flowoling cell

In [20]:
cb = cbfs()


inp = pn.widgets.TextInput( placeholder='Enter text here…', width=600,styles={'background-color': '#F6F6F6'})

conversation = pn.bind(cb.convchain, inp)


tab1 = pn.Column(
    pn.Row(inp),
    pn.layout.Divider(),

    pn.panel(conversation,  loading_indicator=True ,height_policy="fit",styles={'background-color': '#F6F6F6'}),
    pn.layout.Divider()

)

dashboard = pn.Column(
    pn.Row(pn.pane.Markdown('# ChatWithYourData_Bot'),styles={'background-color': '#F6F6F6'},width=672),
    pn.Tabs(('Conversation', tab1),styles={'background-color': '#F6F6F6'})
)
dashboard

Column
    [0] Row(styles={'background-color': '...}, width=672)
        [0] Markdown(str)
    [1] Tabs(styles={'background-color': '...})
        [0] Column
            [0] Row
                [0] TextInput(placeholder='Enter text here…', styles={'background-color': '...}, width=600)
            [1] Divider()
            [2] ParamFunction(function, _pane=WidgetBox, defer_load=False, height_policy='fit', loading_indicator=True, styles={'background-color': '...})
            [3] Divider()

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
